# How to use regioinvent

To be able to use regioinvent, we will need:
- to install the brightway2 Python library (brightway2 and NOT brightway2.5)
- a brightway project within which there is an ecoinvent database with the version 3.9.1 cut-off (only supported version so far)
- to download the trade database stored here: https://doi.org/10.5281/zenodo.11583815 (was extracted from the UN COMTRADE database)

In [ ]:
import pandas as pd
import numpy as np
import sys
# change the path here to wherever you stored the Regioinvent Python package
sys.path.append('C://Users/11max/PycharmProjects/Regioinvent/')
import regioinvent

The initialization of the Regioinvent class requires 4 arguments:
- the path to the trade database
- the name you want to to give to the resulting regionalized ecoinvent database created
- the name of your brightway2 project
- the name of the ecoinvent v3.9.1 cut-off database in your brightway2 project
- a boolean if you wish to regionalize elementary flows or not (True by default)
- a cutoff value to determine how "deep" regionalization should go. 0.95 cutoff leads to a ~120,000 processes database, cutoff at 0.99 leads to a ~200,000 processes database 

In [ ]:
regio = regioinvent.Regioinvent(trade_database_path='C://Users/11max/PycharmProjects/Regioinvent/trade_data.db', 
                                regioinvent_database_name='Regioinvent', 
                                bw_project_name='ecoinvent3.9', 
                                ecoinvent_database_name='ecoinvent3.9.1 cut-off',
                                regionalized_elementary_flows=True,
                                cutoff=0.95)

The initialization will first import everything related to regionalized elementary flows (if regionalized_elementary_flows=True), thus creating two new databases: a regionalized biosphere3 and a regionalized version of ecoinvent (only for elementary flows), as well as importing the regionalized version of the IMPACT World+ LCIA methodology. <br> The created regionzalized version of ecoinvent also accounts for the creation of new water technosphere flows to correct inconsistencies in ecoinvent processes using water technosphere flows. <br>
It will then use the ```wurst``` Python library to extract your ecoinvent database from brightway into a list of dictionnary format which tremendously speed up operations made to the database.

Next, we extract and format the export data from the trade database

In [ ]:
regio.format_export_data()

We estimate the domestic production data based on the ratio of domestic vs export of the EXIOBASEv3.8.2 database 2019. This ratio is applied to self.export_data.

In [ ]:
regio.estimate_domestic_production()

We apply first order regionalization, that is for the 1870 traded products of the ecoinvent3.9.1 cut-off database, we determine the exporters of each product (through self.export_data) and then regionalize the following key inputs:
- electricity
- municipal solid waste
- heat

In [ ]:
regio.first_order_regionalization()

We then extract and format the import data from the trade database

In [ ]:
regio.format_import_data()

With the domestic production and import data in hand, we can determine consumption markets for each of the 1870 products.

In [ ]:
regio.create_consumption_markets()

We relink created consumption markets to regionalized processes across the regioinvent database.

In [ ]:
regio.second_order_regionalization()

We regionalize the elementary flows (if the corresponding boolean was selected)

In [ ]:
regio.regionalize_elem_flows()

Write regioinvent processes in the brightway2 SQL database.

In [ ]:
regio.write_regioinvent_to_database()

We reconnect ecoinvent processes to existing regioinvent processes, so that, e.g., electricity production, coal, CA uses the consumption market of coal for Canada that was created.

In [ ]:
regio.connect_ecoinvent_to_regioinvent()

Done!! You can now go on brightway2 or AB to perform your LCAs normally.